<a href="https://colab.research.google.com/github/Rpmalukani22/Personalized-Medicine-Redefining-Cancer-Treatment/blob/master/Personalized_Medicine_Redefining_Cancer_Treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [10]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [0]:
#verification
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

In [12]:
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 5.3MB 15.0MB/s 
     |████████████████████████████████| 112kB 50.1MB/s 
     |████████████████████████████████| 112kB 53.4MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 


In [13]:
!kaggle competitions download -c msk-redefining-cancer-treatment

  0% 0.00/24.2k [00:00<?, ?B/s]
100% 24.2k/24.2k [00:00<00:00, 20.6MB/s]
  0% 0.00/47.5k [00:00<?, ?B/s]
100% 47.5k/47.5k [00:00<00:00, 42.6MB/s]
 80% 49.0M/61.0M [00:02<00:00, 15.3MB/s]
100% 61.0M/61.0M [00:02<00:00, 28.2MB/s]
 82% 81.0M/99.0M [00:01<00:00, 27.7MB/s]
100% 99.0M/99.0M [00:02<00:00, 51.6MB/s]
  0% 0.00/765 [00:00<?, ?B/s]
100% 765/765 [00:00<00:00, 804kB/s]
  0% 0.00/7.25k [00:00<?, ?B/s]
100% 7.25k/7.25k [00:00<00:00, 7.82MB/s]
 56% 5.00M/8.88M [00:00<00:00, 21.8MB/s]
100% 8.88M/8.88M [00:00<00:00, 35.1MB/s]
  0% 0.00/1.28k [00:00<?, ?B/s]
100% 1.28k/1.28k [00:00<00:00, 1.36MB/s]
  0% 0.00/592 [00:00<?, ?B/s]
100% 592/592 [00:00<00:00, 178kB/s]


In [14]:
!ls -l

total 172980
-rw-r--r-- 1 root root        67 Oct 29 13:05 kaggle.json
drwxr-xr-x 1 root root      4096 Oct 25 16:58 sample_data
-rw-r--r-- 1 root root      1307 Oct 29 13:06 stage1_solution_filtered.csv.7z
-rw-r--r-- 1 root root       592 Oct 29 13:06 stage_2_private_solution.csv.7z
-rw-r--r-- 1 root root       765 Oct 29 13:06 stage2_sample_submission.csv.7z
-rw-r--r-- 1 root root   9308786 Oct 29 13:06 stage2_test_text.csv.7z
-rw-r--r-- 1 root root      7422 Oct 29 13:06 stage2_test_variants.csv.7z
-rw-r--r-- 1 root root 103787620 Oct 29 13:06 test_text.zip
-rw-r--r-- 1 root root     48614 Oct 29 13:06 test_variants.zip
drwxr-xr-x 2 root root      4096 Oct 29 12:59 training
-rw-r--r-- 1 root root  63917183 Oct 29 13:06 training_text.zip
-rw-r--r-- 1 root root     24831 Oct 29 13:06 training_variants.zip


In [15]:
!mkdir -p training
!unzip training_variants.zip -d ./training
!ls -l ./training

Archive:  training_variants.zip
  inflating: ./training/training_variants  
total 68
-rw-rw-r-- 1 root root 66688 Jun 22  2017 training_variants


In [0]:
import pandas as pd

In [0]:
df=pd.read_csv('training/training_variants')

In [22]:
df.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4
